In [64]:
import pandas
#pandas.set_option("display.max_rows", 50, "display.max_columns", 50)

import numpy
import pygsheets
import datetime
import re

In [65]:
#pandas.io.formats.excel.ExcelFormatter.header_style = None
excel_writer = pandas.ExcelWriter('GGIT-Pipelines-SummarySheets-'+str(datetime.date.today())+'.xlsx', engine='xlsxwriter')

import Pipelines_Current dataset

In [66]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
#spreadsheet = gc.open_by_key('18od3L-Uy-N0K0mnUIaeBWLVmZKeaLnjG9rOVSOjSSy4') # Jan 2022?
spreadsheet = gc.open_by_key('1bfPrp0w8Ruorq08Qe4hD8M3xVJ5e00phZ6ApFivO98k') # Dec 2022

#spreadsheet[1] "Gas Pipelines" tab is the second index
gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A2')
#owners = spreadsheet[2].get_as_df()

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)

# delete columns that aren't the same in the sheets, to concatenate them...
columns_not_in_oil = list(set(gas_pipes.columns)-set(oil_pipes.columns))
columns_not_in_gas = list(set(oil_pipes.columns)-set(gas_pipes.columns))
gas_pipes.drop(columns=columns_not_in_oil, axis=1, inplace=True)
oil_pipes.drop(columns=columns_not_in_gas, axis=1, inplace=True)

pipes_df_orig = gas_pipes.copy() #pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
#owners_df_orig = spreadsheet[3].get_as_df(encoding='latin1')
#owners_df_orig.set_index('ProjectID', inplace=True)

# remove empty cells for pipes, owners
#pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
#owners_df_orig = owners_df_orig[owners_df_orig.index!='']

In [67]:
country_ratios_df.replace('--', numpy.nan, inplace=True)

pipes_df_orig.replace('--',numpy.nan,inplace=True)

# remove empty cells for pipes, owners
pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
pipes_df_orig = pipes_df_orig[pipes_df_orig['Route']!='N/A']

### create dataframes for region, country_ratios_df, owners_df

In [68]:
country_ratios_df_touse = country_ratios_df.copy()
pipes_df_touse = pipes_df_orig.copy()

### sum MergedKmByCountry and MergedKmByRegion

In [69]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(list(set(country_ratios_df_touse['Region'])))

In [70]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']

In [71]:
fuel_type = 'Gas'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

country_ratios_df_subset = country_ratios_df_touse.copy()[country_ratios_df_touse.Fuel==fuel_type]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby('Region')['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.name = 'Region'
km_by_country.index.name = 'Country'

totals_row = km_by_region.sum(axis=0)
totals_row.name = 'Total'
km_by_region = km_by_region.append(totals_row)

totals_row = km_by_country.sum(axis=0)
totals_row.name = 'Total'
km_by_country = km_by_country.append(totals_row)

km_by_region.to_excel(excel_writer, sheet_name='Pipeline km by region, status')
km_by_country.to_excel(excel_writer, sheet_name='Pipeline km by country, status')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_75750/2813645818.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_region = km_by_region.append(totals_row)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_75750/2813645818.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_country = km_by_country.append(totals_row)


# parent analysis

In [72]:
owner_parent_calculations_df = pandas.DataFrame()

for idx,row in pipes_df_orig.iterrows():
    parent_string = row.Parent
    parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have a Owner [%] syntax
    percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    if parent_list.__len__()!=percent_list.__len__():
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing
    for p_idx,parent in enumerate(parent_list):
        #print(percent_list[p_idx])
        #print(row.LengthMergedKm)
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ProjectID':row.ProjectID, 
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'LengthMergedKm':row.LengthMergedKm}])])

owner_parent_calculations_df['KmOwnership'] = owner_parent_calculations_df['FractionOwnership']*owner_parent_calculations_df['LengthMergedKm']

In [73]:
owner_parent_calculations_df.LengthMergedKm.astype(float)

0     217.60
0     217.60
0    2111.46
0       0.00
0    1287.00
      ...   
0      74.10
0     543.87
0     432.72
0    1153.00
0       4.83
Name: LengthMergedKm, Length: 5116, dtype: float64

# km by parent company, status

In [74]:
unique_owner_list = owner_parent_calculations_df.Parent.sort_values().unique().tolist()

##################################################
# create km count by owner, status
##################################################
owners_km_by_status_df = pandas.DataFrame(0.0, index=unique_owner_list, columns=status_list)

for status in status_list:
    
    pipes_df_temporary = pipes_df_orig.loc[(pipes_df_orig.Status==status)&
                                           (pipes_df_orig.Fuel=='Gas')]
    op_temporary = owner_parent_calculations_df.loc[owner_parent_calculations_df.ProjectID.isin(pipes_df_temporary.ProjectID)]
    op_sum = pandas.DataFrame(op_temporary.groupby('Parent', dropna=False)['KmOwnership'].sum(min_count=0))
    owners_km_by_status_df.loc[:,status] = op_sum

owners_km_by_status_df.index.name = 'Parent Company'
owners_km_by_status_df['In Development (Proposed + Construction)'] = owners_km_by_status_df[['Proposed','Construction']].sum(axis=1, min_count=0)
owners_km_by_status_df = owners_km_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list]

totals_row = owners_km_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_km_by_status_df = owners_km_by_status_df.append(totals_row)

owners_km_by_status_df.replace(numpy.nan, 0, inplace=True)
owners_km_by_status_df = owners_km_by_status_df.loc[~(owners_km_by_status_df==0).all(axis=1)]

owners_km_by_status_df.to_excel(excel_writer, sheet_name='Pipeline km by company')

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_75750/2413215303.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  owners_km_by_status_df = owners_km_by_status_df.append(totals_row)


In [75]:
owners_km_by_status_df.tail()

,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Parent Company,,,,,,,,,
珠江三角洲投资有限公司,0.000,0.0000,0.0000,0.0000,0.000,66.15000,0.0,0.00,0.00
辽河油田燃气集团公司,0.000,0.0000,0.0000,0.0000,0.000,168.00000,0.0,0.00,0.00
香港中华煤气（河北）有限公司,79.550,174.3240,253.8740,0.0000,0.000,199.09000,0.0,0.00,0.00
（香港）中国燃气控股有限公司,42.480,142.8910,185.3710,0.0000,115.394,50.96000,0.0,0.00,0.00
Total,152794.143,59552.9461,212347.0891,37685.4341,56825.430,969613.89574,331.0,9364.73,2721.58


# pipeline km by start year, type

In [76]:
pipes_started_df = pipes_df_touse.copy()
#pipes_started_eu['StartYearLatest'].replace(numpy.nan,'',inplace=True)

pipes_started_df = pipes_started_df[(pipes_started_df['Status'].isin(['Operating'])) &
                              (pipes_started_df['Fuel']=='Gas')]
pipes_started_df_sum = pipes_started_df.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [77]:
km_by_start_year = pandas.DataFrame(index=list(range(1980,2023)), columns=['Gas pipeline km'])
km_by_start_year.index.name = 'Start year'

km_by_start_year['Gas pipeline km'] = pipes_started_df_sum
km_by_start_year.replace(numpy.nan,0,inplace=True)

totals_row = km_by_start_year.sum(axis=0)
totals_row.name = 'Total'
km_by_start_year = km_by_start_year.append(totals_row)

km_by_start_year.to_excel(excel_writer, sheet_name='Pipeline km by start year')

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_75750/3035969754.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  km_by_start_year = km_by_start_year.append(totals_row)


# save excel file

In [78]:
excel_writer.save()

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_75750/2638382022.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
